# **Schema**
- Nuts and Bolts of working with Large Language Models (LLMs)

**Problem Statement**
- Design and implement a schema for interacting with Large Language Models (LLMs) that ensures consistent, structured, and efficient communication.

## **Text**
- The natural language way to interact with LLMs

In [4]:
%%capture
# update or install the necessary libraries
%pip install --upgrade langchain_core langchain_community langchain_aws python-dotenv

In [5]:
%pip show langchain_core

Name: langchain-core
Version: 1.0.1
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, pyyaml, tenacity, typing-extensions
Required-by: langchain, langchain-aws, langchain-classic, langchain-community, langchain-mcp-adapters, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_aws import ChatBedrock
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION")

In [10]:
# You'll be working with simple strings as prompts(that'll soon grow in complexity!)
my_text = "What day comes after Friday?"
my_text

'What day comes after Friday?'

In [11]:
from langchain_core.messages import HumanMessage
chat = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.7,
)
response = chat.invoke([HumanMessage(content=my_text)])
response.content

' The day that comes after Friday is Saturday. In a weekly cycle, Friday is followed by Saturday.'

In [26]:
response

AIMessage(content=' The document is a personal one labeled "my document." Its content consists of text collected from various sources. There is no specific information or topic presented in the document. A summary is not possible as the document does not contain original or substantial content.', additional_kwargs={'usage': {'prompt_tokens': 47, 'completion_tokens': 50, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 97}, 'stop_reason': None, 'thinking': {}, 'model_id': 'mistral.mistral-7b-instruct-v0:2'}, response_metadata={'usage': {'prompt_tokens': 47, 'completion_tokens': 50, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 97}, 'stop_reason': None, 'thinking': {}, 'model_id': 'mistral.mistral-7b-instruct-v0:2', 'model_provider': 'bedrock', 'model_name': 'mistral.mistral-7b-instruct-v0:2'}, id='lc_run--9e85a976-41db-4baf-8750-e55911eb9e02-0', usage_metadata={'input_tokens': 47, 'output_tokens': 50, 'total_tokens': 97, 'input_

## **Chat Messages**
Like text, but specified with a message type (System, Human, AI)

* **System** - Helpful background context that tell the AI what to do
* **Human** - Messages that are intented to represent the user
* **AI** - Messages that show what the AI responded with

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

Now let's create a few messages that simulate a chat experience with a bot


In [14]:
# Chat messages example
response = chat.invoke([
    SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
    HumanMessage(content="I like tomatoes, what should I eat?")
])
print(response.content)

 You could try a fresh tomato salad or a sandwich with tomatoes for a tasty and nutritious meal.



You can also pass more chat history w/ responses from the AI

In [16]:
response = chat.invoke(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)
response.content

'\nExplore the Old Town, visit the Colline du Château, and enjoy the local cuisine.\n[\n]()\n\n[</SYS>](mailto:youremail@example.com?subject=Recommendations%20for%20Nice%2C%20France&body=Hi%2C%0A%0A%0ASome%20recommended%20activities%20for%20Nice%2C%20France%20are:%0A%0A-%20Explore%20the%20Old%20Town%20(%0252CVieille%20Ville%252C%20Nice%2C%20France%29%2C%20with%20its%20narrow%20streets%2C%20colorful%20buildings%2C%20and%20unique%20shops%2C%20cafes%2C%20and%20restaurants%20-%0A%0A-%20Visit%20the%20Colline%20du%20Ch%C3%A2te%20(%0252CCastle%20Hill%2C%20Nice%2C%20France%29%2C'

You can also exclude the system message if you want

In [ ]:
response = chat.invoke(
    [
        HumanMessage(content="What day comes after Thursday?")
    ]
)
response.content

## **Documents**
An object that holds a piece of text and metadata (more information about that text)

In [19]:
from langchain_core.documents import Document

In [20]:
document = Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "The LangChain Papers",
             'my_document_create_time' : 1680013019
         })

In [21]:
# Prepare the prompt by combining the document content with your custom prompt
prompt = f"Document Content: {document.page_content}\n\nPlease summarize the above document."

In [27]:
prompt

"Document Content: This is my document. It is full of text that I've gathered from other places\n\nPlease summarize the above document."

In [22]:
# Get the response from the language model
response = chat.invoke([HumanMessage(content=f"Summarize the following document:\n\n{prompt}")])
response.content

' The document is a personal one labeled as "my document." Its content consists of text that the document\'s owner has collected from various sources. No specific information or topics can be extracted from the document as it doesn\'t provide any context or details beyond being a container for borrowed text.'

But you don't have to include metadata if you don't want to

In [23]:
document1 = Document(page_content="This is my document. It is full of text that I've gathered from other places")

In [24]:
# Prepare the prompt by combining the document content with your custom prompt
prompt1 = f"Document Content: {document1.page_content}\n\nPlease summarize the above document."

In [25]:
# Get the response from the language model
response = chat.invoke([HumanMessage(content=f"Summarize the following document:\n\n{prompt1}")])
response.content

' The document is a personal one labeled "my document." Its content consists of text collected from various sources. There is no specific information or topic presented in the document. A summary is not possible as the document does not contain original or substantial content.'

# **Let's Do an Activity**

Create a simple interaction using LangChain and AWS. Define system, human, and AI messages to simulate a conversation with a travel recommendation bot. Additionally, create a document schema to store information about travel destinations.

**Steps**

* **Set Up**: Install the necessary libraries and set up your AWS Secret Key and Access Key.
* **Define Messages**: Create system, human, and AI messages for a travel bot.
* **Simulate Interaction**: Use the chat function to simulate a conversation.
* **`Create Document`**: Define a document schema to store information about a travel destination.